In [51]:
!pip install openai==0.28.1
!pip install fpdf==1.7.2


In [52]:
import pandas as pd
import openai
import time
from fpdf import FPDF
from google.colab import userdata

openai.api_key = userdata.get("OPENAI_API_KEY")

In [53]:
df = pd.read_csv('https://raw.githubusercontent.com/11andrea2233/RAG-Healthcare-Database/refs/heads/main/Database.csv')

In [54]:
df.sample(5)

,Patient ID,First Name,Last Name,DOB,Gender,Address,Phone Number,Emergency Contact,Smoking Status,Alcohol Consumption,...,Condition(s),Allergy,Medications,BP (mmHg),BMI,Last Visit,Diagnosis Code,Treatment,Insurance Provider,Amount Billed
4,P0005,Alice,Wong,1/5/2002,F,ZIP 78901,555-6789,Nancy Wong,Non-Smoker,NaN,...,NaN,NaN,NaN,110/70,22,11/5/2024,Z00.00,General Checkup,Youth Health,$80
1,P0002,Sarah,Lee,7/20/1982,F,ZIP 67890,555-5678,Mike Lee,Smoker,Heavy,...,Diabetes Type 2,Latex,Metformin 500mg,140/90,32,10/25/2024,E11,Diabetes Mgmt,Wellness Plan,$200
7,P0008,Robert,Davis,9/25/1978,M,ZIP 66778,555-2222,Ann Davis,Ex-Smoker,Moderate,...,COPD,NaN,Albuterol Inhaler,130/80,29,11/6/2024,J44.9,Respiratory Mgmt,CarePlus,$175
16,P0017,Lily,Martinez,12/11/1973,F,ZIP 99877,555-1212,Max Martinez,Non-Smoker,Light,...,Hypertension,NaN,Lisinopril 10mg,130/85,27,11/6/2024,I10,Blood Pressure Check,Medicare,$160
5,P0006,Michael,Brown,4/12/1985,M,ZIP 11223,555-1010,Lisa Brown,Smoker,Moderate,...,"Hypertension, Diabetes",Sulfa,Lisinopril 10mg,150/95,30,11/2/2024,"E11, I10",Chronic Care Mgmt,Medicare,$250


In [55]:
column_info = df.dtypes
print(column_info, "\n")

Patient ID             object
First Name             object
Last Name              object
DOB                    object
Gender                 object
Address                object
Phone Number           object
Emergency Contact      object
Smoking Status         object
Alcohol Consumption    object
Exercise Habits        object
Condition(s)           object
Allergy                object
Medications            object
BP (mmHg)              object
BMI                     int64
Last Visit             object
Diagnosis Code         object
Treatment              object
Insurance Provider     object
Amount Billed          object
dtype: object 



In [56]:
# Identify Categorical and Numerical Features
categorical_features = df.select_dtypes(include=['object']).columns
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns

print("Categorical Features:", categorical_features)
print("Numerical Features:", numerical_features)

Categorical Features: Index(['Patient ID', 'First Name', 'Last Name', 'DOB', 'Gender', 'Address',
       'Phone Number', 'Emergency Contact', 'Smoking Status',
       'Alcohol Consumption', 'Exercise Habits', 'Condition(s)', 'Allergy',
       'Medications', 'BP (mmHg)', 'Last Visit', 'Diagnosis Code', 'Treatment',
       'Insurance Provider', 'Amount Billed'],
      dtype='object')
Numerical Features: Index(['BMI'], dtype='object')


In [57]:
# Get Unique Values for Categorical Features with Limitation
def get_unique_values(col, max_display=10):
    uniques = df[col].unique().tolist() # get unique values and turn them into lists
    if len(uniques) > max_display: # if there are more than 10 unique values
        return uniques[:max_display] + ["..."]
    return uniques

unique_values = {col: get_unique_values(col) for col in categorical_features}

print("Unique Values for Categorical Features:")
for col, uniques in unique_values.items():
    print(f" - {col}: {uniques}")
print()

Unique Values for Categorical Features:
 - Patient ID: ['P0001', 'P0002', 'P0003', 'P0004', 'P0005', 'P0006', 'P0007', 'P0008', 'P0009', 'P0010', '...']
 - First Name: ['John', 'Sarah', 'David', 'Maria', 'Alice', 'Michael', 'Emma', 'Robert', 'Chloe', 'Ethan', '...']
 - Last Name: ['Doe', 'Lee', 'Kim', 'Perez', 'Wong', 'Brown', 'Smith', 'Davis', 'White', 'Hall', '...']
 - DOB: ['2/14/1975', '7/20/1982', '3/15/1990', '12/30/1967', '1/5/2002', '4/12/1985', '8/30/1999', '9/25/1978', '3/10/1987', '6/15/2000', '...']
 - Gender: ['M', 'F']
 - Address: ['ZIP 12345', 'ZIP 67890', 'ZIP 54321', 'ZIP 23456', 'ZIP 78901', 'ZIP 11223', 'ZIP 44556', 'ZIP 66778', 'ZIP 33445', 'ZIP 55667', '...']
 - Phone Number: ['555-1234', '555-5678', '555-9012', '555-3456', '555-6789', '555-1010', '555-1111', '555-2222', '555-3333', '555-4444', '...']
 - Emergency Contact: ['Jane Doe', 'Mike Lee', 'Emily Kim', 'Alex Perez', 'Nancy Wong', 'Lisa Brown', 'Oliver Smith', 'Ann Davis', 'Tom White', 'Mia Hall', '...']
 - 

In [58]:
# Get Descriptive Statistics for Numerical Features
numerical_stats = df[numerical_features].describe()
print("Descriptive Statistics for Numerical Features:\n", numerical_stats, "\n")

Descriptive Statistics for Numerical Features:
              BMI
count  20.000000
mean   27.800000
std     3.679245
min    22.000000
25%    24.750000
50%    28.500000
75%    30.000000
max    35.000000 



In [59]:
def generate_template(df,
                      column_info,
                      categorical_features,
                      numerical_features,
                      unique_values,
                      numerical_stats):

    # Construct a summary of the dataframe's structure
    column_summary = "Column Names and Data Types:\n"
    for col, dtype in column_info.items():
        column_summary += f" - {col}: {dtype}\n"

    # Unique values for categorical features
    unique_values_str = "Unique Values for Categorical Features:\n"
    for col, uniques in unique_values.items():
        unique_values_str += f" - {col}: {uniques}\n"

    # Descriptive statistics for numerical features
    numerical_stats_str = "Descriptive Statistics for Numerical Features:\n"
    for col in numerical_features:
        numerical_stats_str += f" - {col}:\n"
        for stat_name, value in numerical_stats[col].items():
            numerical_stats_str += f"   {stat_name}: {value}\n"

    # Define the system prompt
    system_prompt = """You are an intelligent assistant that creates descriptive templates for transforming dataframe rows into coherent paragraphs.
    Analyze the provided dataframe structure and generate a template sentence that includes placeholders for each column.
    Ensure the template is contextually relevant and maintains grammatical correctness."""

    # Define the user prompt
    user_prompt = user_prompt = f"""
    Analyze the following dataframe structure and create a descriptive template with placeholders for each column.

    <column_summary>
    {column_summary}
    </column_summary>

    <unique_values>
    {unique_values_str}
    </unique_values>

    <numerical_stats>
    {numerical_stats_str}
    </numerical_stats>

    Use the exact column names from the column_summary in generating the variable names in the template,
    as they will be populated with the actual values in the dataset.

    Example Template about a Spotify dataset:
    "{{artist}} gained {{streams}} streams in the song '{{song}}' that was a hit in {{date}}."

    Output only the template without any explanation or introduction.
    The template's variables will be dynamically replaced so make sure they're formatted properly."""

    # Generate the template (with retries)
    retries = 3
    for attempt in range(retries):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                temperature=0.3,
                max_tokens=1024,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            )
            template = response['choices'][0]['message']['content'].strip()
            return template
        except Exception as e:
            print(f"Error generating template (Attempt {attempt + 1}/{retries}): {e}")
            time.sleep(2)  # Wait before retrying

    return None

In [60]:
# Generate the template
template = generate_template(df,
                             column_info,
                             categorical_features,
                             numerical_features,
                             unique_values,
                             numerical_stats)

print(template)

"Patient {Patient ID}, {First Name} {Last Name}, born on {DOB}, identifies as {Gender} and resides at {Address}. They can be reached at {Phone Number} and their emergency contact is {Emergency Contact}. Currently, their smoking status is {Smoking Status}, and they report {Alcohol Consumption} alcohol consumption along with {Exercise Habits} exercise habits. The patient has the following condition(s): {Condition(s)}, and is allergic to {Allergy}. They are currently taking the following medications: {Medications}. Their blood pressure reading is {BP (mmHg)} and their BMI is {BMI}. The last visit was on {Last Visit}, with a diagnosis code of {Diagnosis Code} and the treatment provided was {Treatment}. The insurance provider for this patient is {Insurance Provider}, and the amount billed was {Amount Billed}."


In [61]:
def populate_template(template, row):

    # Convert row to dictionary and replace NaN with 'N/A'
    row_dict = row.to_dict()
    for key, value in row_dict.items():
        if pd.isna(value):
            row_dict[key] = 'N/A'

    # Generate the populated template per row
    paragraph = template.format(**row_dict)

    return paragraph

In [62]:
# Apply the populate_template function to each row in the dataframe
df['paragraph'] = df.apply(lambda row: populate_template(template, row), axis=1)

In [63]:
df['paragraph'][0]

'"Patient P0001, John Doe, born on 2/14/1975, identifies as M and resides at ZIP 12345. They can be reached at 555-1234 and their emergency contact is Jane Doe. Currently, their smoking status is Non-Smoker, and they report Moderate alcohol consumption along with Regular (3x/wk) exercise habits. The patient has the following condition(s): Hypertension, and is allergic to Penicillin. They are currently taking the following medications: Atenolol 50mg. Their blood pressure reading is 130/85 and their BMI is 28. The last visit was on 11/1/2024, with a diagnosis code of I10 and the treatment provided was Blood Pressure Check. The insurance provider for this patient is HealthCare Inc, and the amount billed was $150 ."'

In [64]:
def save_content_to_txt(paragraphs, txt_filename):
    try:
        with open(txt_filename, 'w', encoding='utf-8') as file:

            for para in paragraphs:
                file.write(para + '\n\n')  # Double newline for separation

        print(f"All content has been saved as '{txt_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to text file: {e}")

In [65]:
def save_content_to_pdf(paragraphs, pdf_filename):
    try:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.set_font("Arial", size=12)

        # Add each paragraph
        for para in paragraphs:
            pdf.multi_cell(0, 10, para)
            pdf.ln()  # Add a line break between paragraphs

        pdf.output(pdf_filename)
        print(f"All content has been saved as '{pdf_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to PDF: {e}")

In [66]:
# Save the paragraphs to both text and PDF files
save_content_to_txt(df['paragraph'].to_list(),
                    txt_filename='nlg.txt')

save_content_to_pdf(df['paragraph'].to_list(),
                    pdf_filename='nlg.pdf')

All content has been saved as 'nlg.txt'.

All content has been saved as 'nlg.pdf'.

